# Jupyter Notebook: Tech Challenge - Fine-Tuning a Foundation Model

Este notebook implementa o Tech Challenge da fase 3, que consiste em realizar o fine-tuning de um modelo foundational utilizando o dataset "AmazonTitles-1.3MM". O objetivo é treinar o modelo para responder perguntas dos usuários com base nos títulos e descrições de produtos do dataset.

In [ ]:
# ============================
# 1. Instalar Dependências
# ============================
!pip install -q unsloth accelerate peft trl bitsandbytes transformers datasets

In [21]:
!pip uninstall -y bitsandbytes
!pip install bitsandbytes --prefer-binary --upgrade --no-cache-dir
import bitsandbytes as bnb
print(bnb.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 107.8 MB/s eta 0:00:00


0.45.4


In [3]:


# ============================
# 2. Carregar e Preparar Dataset
# ============================
import json
import pandas as pd

# Caminho para o dataset
dataset_path = "trn-amostra.jsonl"

# Carregar dados
with open(dataset_path, 'r') as f:
    lines = f.readlines()
    data = [json.loads(line) for line in lines]

# Criar DataFrame
df = pd.DataFrame(data)

# Filtrar entradas com descrição
df = df[df["content"].str.strip() != ""]

print(df)
# Criar estrutura de prompt/response
df_prompt = pd.DataFrame({
    "prompt": df["title"].apply(lambda x: f"Pergunta do usuário sobre o produto: {x}"),
    "response": df["content"]
})

df_prompt


              uid                                              title  \
0      0000031909                        Girls Ballet Tutu Neon Pink   
3      0001360000                                      Mog's Kittens   
7      0000031895                        Girls Ballet Tutu Neon Blue   
12     000100039X                                        The Prophet   
13     0001473905                          Rightly Dividing the Word   
...           ...                                                ...   
10111  0061579556  The Web and The Root (Harper Perennial Modern ...   
10112  0061584991                           Hardly Knew Her: Stories   
10113  0061626015  Discovering God: The Origins of the Great Reli...   
10114  0061577936  The Department of Mad Scientists: How DARPA Is...   
10115  0061579084  The Vision Board: The Secret to an Extraordina...   

                                                 content  \
0      High quality 3 layer ballet tutu. 12 inches in...   
3      Judith K

,prompt,response
0,Pergunta do usuário sobre o produto: Girls Bal...,High quality 3 layer ballet tutu. 12 inches in...
3,Pergunta do usuário sobre o produto: Mog's Kit...,Judith Kerr&#8217;s best&#8211;selling adventu...
7,Pergunta do usuário sobre o produto: Girls Bal...,Dance tutu for girls ages 2-8 years. Perfect f...
12,Pergunta do usuário sobre o produto: The Prophet,"In a distant, timeless place, a mysterious pro..."
13,Pergunta do usuário sobre o produto: Rightly D...,--This text refers to thePaperbackedition.
...,...,...
10111,Pergunta do usuário sobre o produto: The Web a...,&#8220;Wolfe wrote as one inspired. No one of ...
10112,Pergunta do usuário sobre o produto: Hardly Kn...,Fans of bestseller Lippman's long-running seri...
10113,Pergunta do usuário sobre o produto: Discoveri...,*Starred Review* Skeptics such as Richard Dawk...
10114,Pergunta do usuário sobre o produto: The Depar...,If you want to know who really invented the In...


In [12]:
# ============================
# 3. Converter para Hugging Face Dataset
# ============================
from datasets import Dataset
hf_dataset = Dataset.from_pandas(df_prompt)

# ============================
# 4. Carregar modelo com Unsloth
# ============================
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

# Preparar modelo para PEFT
model = FastLanguageModel.get_peft_model(model)



==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [13]:
# ============================
# 5. Testar modelo antes do Fine-tuning
# ============================
input_text = "Pergunta do usuário sobre o produto: Girls Ballet Tutu Neon Pink"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Pergunta do usuário sobre o produto: Girls Ballet Tutu Neon Pink
A tutu é uma vestimenta infantil que, geralmente, é feita de uma malha e de um fio de seda. Ela é normalmente usada como parte de um vestido, um manto, ou uma roupa. Em geral, é feita de seda ou de algodão, e pode ser pintado ou colorido. Ela é normalmente usada como parte de um vestido, um manto, ou uma rou


In [14]:

# ============================
# 6. Preparar Tokenização para treinamento
# ============================
def formatting_func(example):
    return f"<s>[INST] {example['prompt']} [/INST] {example['response']} </s>"

def tokenize_func(example):
    formatted = formatting_func(example)
    tokenized = tokenizer(
        formatted,
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Tokenizar o dataset
tokenized_dataset = hf_dataset.map(tokenize_func)



Map:   0%|          | 0/7642 [00:00<?, ? examples/s]

In [11]:
ex = tokenized_dataset[0]
print(tokenizer.decode(ex['input_ids']))

<|begin_of_text|><s>[INST] Pergunta do usuário sobre o produto: Girls Ballet Tutu Neon Pink [/INST] High quality 3 layer ballet tutu. 12 inches in length </s><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_

In [ ]:
# ============================
# 7. Treinar modelo (Fine-tuning)
# ============================
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="llama-finetuned",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    logging_dir="logs",
    logging_steps=5,
    save_steps=100,
    save_total_limit=1,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Iniciar treinamento
trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,642 | Num Epochs = 1 | Total steps = 1,911
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)


Step,Training Loss
5,3.169300
10,2.835500
15,2.767300
20,2.561000
25,2.444700
30,2.737200
35,2.688700
40,2.638000
45,2.542800
50,2.605500


In [ ]:

# ============================
# 8. Testar modelo após Fine-tuning
# ============================

# Entrada do usuário (título do produto)
pergunta = "Pergunta do usuário sobre o produto: Girls Ballet Tutu Neon Pink"

# Tokenizar e gerar resposta
inputs = tokenizer(pergunta, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=150)
resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Exibir resultado com formatação clara
print("="*60)
print("🟡 PERGUNTA DO USUÁRIO:")
print(pergunta)
print("\n🟢 RESPOSTA DO MODELO:")
print(resposta)
print("="*60)

tensor([[128000,     47,   2431,  43150,    656,  60118,  15482,    297,  44770,
             25,  20666,  87736,  67063,     84,  78064,  27211,    482,   8645,
            220,     17,     51,    482,  20666,  67063,     84,    482,  87736,
            482,  20666,  67063,     84,    482,  78064,  27211,    198,  75588,
          87736,  67063,     84,  78064,  27211,    482,   8645,    220,     17,
             51,    482,  20666,  67063,     84,    482,  87736,    482,  20666,
          67063,     84,    482,  78064,  27211,    198,  75588,  87736,  67063,
             84,  78064,  27211,    482,   8645,    220,     17,     51,    482,
          20666,  67063,     84,    482,  87736,    482,  20666,  67063,     84,
            482,  78064,  27211,    198,  75588,  87736,  67063,     84,  78064,
          27211,    482,   8645,    220,     17,     51,    482,  20666,  67063,
             84,    482,  87736,    482,  20666,  67063,     84,    482,  78064,
          27211,    198,  75